We achieve poor balance using the default algorithms in MatchIt, particularly because the treatment lands tend to be bigger than the control lands. But since we have way more control lands than treatment, and we're interested in the ATT, and since the distinctions between control lands aren't particularly important, we will _aggregate_ control lands together to form superlands to match to the treatment.

This will be done in an approximate, greedy way, with the goal to create 1-1 matches between treatment lands and control superlands.

In [1]:
import pandas as pd
import geopandas as gpd
from pyprojroot.here import here
from annoy import AnnoyIndex
import numpy as np

pd.set_option('display.max_columns', 50)

In [3]:
df = pd.read_csv(here("import/output/com_priv.csv"))
df = df.drop(['Unnamed: 0', 'land_type2'], axis=1)
df = df.set_index(df['id'])

print(len(df))

df.head()

1873211


,treat,area_final,forest1985,temp,precip,elevation,slope,pop_dens,dist_roads,dist_rivers,dist_cities,ephemeral_rest,persistent_rest,id
id,,,,,,,,,,,,,,
1,0,0.038080,0.001081,1.122367,0.909230,0.647723,1.137466,0.161666,0.341784,0.224733,0.272358,0.0,0.000000,1
2,0,0.011091,0.000000,1.122367,0.910020,0.626401,0.564676,0.171645,0.246843,0.060279,0.272358,0.0,0.000000,2
3,0,0.023344,0.000000,1.124045,0.817861,0.569686,0.450653,0.707153,0.046968,0.065659,0.163415,0.0,0.000000,3
4,0,0.003063,0.000000,1.122367,0.819031,0.579695,0.889880,0.707153,0.082221,0.096388,0.163415,0.0,0.000000,4
5,0,0.017007,0.017129,1.137466,0.840758,0.546205,1.202641,0.254516,0.076963,0.267657,0.062253,0.0,2.614985,5


In [374]:
area_col='area_final'
cols_to_sum=['area_final', 'forest1985', 'persistent_rest', 'ephemeral_rest']
outcomes=['persistent_rest', 'ephemeral_rest']
cols_to_drop=['id']

In [351]:
treat = df[df.treat == 1].drop(['treat'] + cols_to_sum + cols_to_drop, axis=1)
treat_with_area = df[df.treat == 1].drop(['treat'], axis=1)

control = df[df.treat == 0].drop(['treat'] + cols_to_sum + cols_to_drop, axis=1)
control_with_area = df[df.treat == 0].drop(['treat'], axis=1)

In [352]:
control_index = AnnoyIndex(control.shape[1], 'euclidean') # Remove the index and the "treatment" columns
for i, (ix, row) in enumerate(control.iterrows()):
    if i % 10000 == 0:
        print('.', end='')
    control_index.add_item(ix, row.values)

control_index.build(10) # 10 trees
control_index.save(str(here("matching/cache/com_priv_control.nn")))

............................................................................................................................................................................................

True

In [271]:
def closest_subset_with_averaging(target, vectors):
    """
    Find a subset of vectors that adheres to custom summation and averaging rules
    to get as close as possible to the target.
    
    Args:
    - target (numpy array): N-dimensional target vector
    - vectors (numpy array): Array of N-dimensional vectors
    
    Returns:
    - list: Indices of the chosen vectors
    """
    current_sum = np.array([0.0] * target.shape[0])
    chosen_indices = []
    
    # Create an array of indices corresponding to the vectors
    indices = np.arange(vectors.shape[0])
    
    while len(vectors) > 0 and not np.allclose(current_sum[:2], target[:2]):
        # Calculate potential sums for the first two components
        potential_sums = vectors.copy()
        potential_sums[:, :2] += current_sum[:2]
        
        # For the 3rd to Nth components, compute the potential weighted averages
        weight_totals = vectors[:, 0] + current_sum[0]
        for i in range(2, target.shape[0]):
            potential_sums[:, i] = (vectors[:, 0] * vectors[:, i] + current_sum[0] * current_sum[i]) / weight_totals
        
        # Calculate distances to target for each potential sum
        distances = np.linalg.norm(potential_sums - target, axis=1)
        best_index = np.argmin(distances)

        # If adding the best vector makes the solution worse, bail out
        if distances[best_index] > np.linalg.norm(current_sum - target):
            break

        # Update the weighted averages for the 3rd to Nth components
        current_sum[2:] = (current_sum[0] * current_sum[2:] + vectors[best_index, 0] * vectors[best_index, 2:]) / (current_sum[0] + vectors[best_index, 0])
        
        # Update current sum for the first two components
        current_sum[:2] += vectors[best_index, :2]
        chosen_indices.append(indices[best_index])
        
        # Remove the selected vector and its index from consideration
        vectors = np.delete(vectors, best_index, axis=0)
        indices = np.delete(indices, best_index, axis=0)

    return chosen_indices
    

In [431]:
# Hacky global variable: this needs to be reset before runs
exclude_ids = set()

def make_superland_for_treatment_land(ta, area_col='area_final', cols_to_sum=['area_final', 'forest1985', 'persistent_rest', 'ephemeral_rest'], outcomes=['persistent_rest', 'ephemeral_rest'], cols_to_drop=['id']):
    ta_use = ta.drop(cols_to_drop + outcomes)
    control_use = control_with_area.drop(cols_to_drop + outcomes, axis=1)
    t = ta_use.drop(cols_to_sum, errors='ignore')
    nearest_results = set(control_index.get_nns_by_vector(t.values, 1000)) - exclude_ids
    nearest = np.array(list(nearest_results))
    candidates = control_use.loc[nearest].to_numpy()
    component_lands = closest_subset_with_averaging(ta_use.values, candidates)

    df_super = control_with_area.loc[nearest[component_lands]]

    # Weighted average of those columns
    df_super_combined = df_super.drop(cols_to_sum + outcomes + cols_to_drop, axis=1).multiply(df_super[area_col], axis='index').sum(axis=0) / df_super[area_col].sum()
    
    # Sum of the remaining columns
    for col in cols_to_sum:
        df_super_combined[col] = df_super[col].sum()

    # Treatment ID
    df_super_combined['treatment_id'] = ta.name

    # Control IDs
    df_super_combined['control_ids'] = df_super.agg({'id': list})['id']
    exclude_ids.update(df_super_combined['control_ids'])
    
    return df_super_combined

make_superland_for_treatment_land(treat_with_area.iloc[1])

In [432]:
exclude_ids = set()
superlands = []

for i, ta in treat_with_area.iterrows():
    if i % 100 == 0:
        print('.', end='')
    result = make_superland_for_treatment_land(ta)
    if result is not None:
        superlands.append((ta, result, sum((ta - result)**2)))

.................

In [435]:
df_superlands = pd.DataFrame([x[1] for x in superlands]).dropna()
len(df_superlands)

1661

In [445]:
df_superlands.control_ids.str.len().value_counts()

control_ids
3      463
2      395
4      276
1      256
5      115
6       58
7       22
9       10
8       10
11       7
19       3
13       3
10       3
12       3
26       2
16       2
44       1
17       1
32       1
461      1
56       1
167      1
21       1
85       1
18       1
377      1
216      1
539      1
869      1
65       1
14       1
46       1
20       1
50       1
70       1
124      1
630      1
66       1
943      1
438      1
29       1
60       1
253      1
245      1
169      1
171      1
569      1
34       1
23       1
Name: count, dtype: int64

In [441]:
from itertools import chain

cids = list(chain(*df_superlands['control_ids'].values))

# For confirming that each control unit are never reused in constructing superlands

print(len(set(cids)), len(cids))

12030 12030


In [446]:
df_superlands.shape

(1661, 14)

In [440]:
df_superlands.to_csv(here("matching/output/com_priv_superlands.csv"), index=False)